In [ ]:
#!python -m pip install pandas
#!python -m pip install seaborn
#!python -m pip install matplotlib

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

#df = sns.load_dataset('titanic_v2.csv')
df = pd.read_csv('titanic_v2.csv')
df

In [ ]:
df.dtypes


In [ ]:
df['sex'] = df['sex'].map({'male': 1, 'female': 2})

In [ ]:
df.describe()

In [ ]:
df['survived'].value_counts()

In [ ]:
korrelaatiomatriisi = df.loc[:,'age':'survived'].corr()


In [ ]:
sns.heatmap(korrelaatiomatriisi, vmin = -1.0, vmax = 1.0, cmap = 'RdBu', annot = True)
plt.show()

In [ ]:
sns.pairplot(df, hue = 'pclass')
plt.show()

3) Datan Valmistelu

In [ ]:
colnames = df.columns
colnames

In [ ]:
X = df.loc[:, 'pclass':'age']
# eotetaan vastemuuttuja talteen series-olioon
y = df.loc[:, 'survived']

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test,y_train , y_test = train_test_split(X, y, 
   test_size = 0.2, random_state=42)
print(X_train.shape)
print(X_test.shape)

In [ ]:
'''from sklearn import tree

classifier = tree.DecisionTreeClassifier(max_depth = 3)
classifier.fit(X, y)'''

# päätöspuuluokitteluja
classifier = tree.DecisionTreeClassifier(max_depth=2)
#sovitetaan muuttujat malliin
classifier.fit(X_train,y_train)

In [ ]:
print(tree.export_text(classifier, feature_names=list(colnames[:3])))

In [ ]:
tree.plot_tree(classifier,
               max_depth=3, 
               feature_names=colnames[:3], 
               class_names=['not_survived', 'survived'], 
               label='all', 
               filled=True, 
               impurity=True, 
               node_ids=True, 
               proportion=True, 
               rounded=True, 
               precision=3, 
               ax=None, 
               fontsize=None)
plt.show()


5) Arviointi

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

#ennuste selittävien muuttujien perusteella
#oikeasti pitäisi tehdä testijoukosta. Tässä sitä ei ole käytetty, koska tietojoukko on niin pieni.
Y_pred = classifier.predict(X_test)

# sekaannusmatriisi
cm = confusion_matrix(y_test, Y_pred)
print("Confusion matrix:\n",cm)

#tarkkuus 
accuracy = accuracy_score(y_test, Y_pred)
print("Accuracy calculated from the training set = %.3f" % (accuracy)) 

#luokitteluraportti
print(classification_report(y_test, Y_pred, target_names=['not_survived','survived']))
'''


Y_pred = classifier.predict(X_test)

cm = confusion_matrix(y_test, Y_pred)
print("Confusion matrix:\n",cm)
accuracy = accuracy_score(y_test, Y_pred)
print("Accuracy calculated from the training set = %.3f (SUBJECT TO OVERFITTING)" % (accuracy))
print(classification_report(y_test, Y_pred, target_names=['no', 'yes']))'''

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
cm = confusion_matrix(y, Y_pred)  #normalize='all'
cmd = ConfusionMatrixDisplay(cm, display_labels=['not_survived','survived'])
cmd.plot()
plt.show()

In [ ]:
from sklearn.model_selection import cross_val_score

# osien määrä, jaetaan k yhtä suureen osaan
k = 2

#arvioinnin tulos kierroksittain
#kullakin kierroksella lasketaan k-1 osaa mallien opettamiseen, 1 osa mallin testaamiseen
#toistetaan k kertaa, lopuksi otetaan kaikkien k-tulosten keskiarvo
scores = cross_val_score(estimator = classifier,
                         X = X_test,
                         y = y_test,
                         scoring = "accuracy",
                         cv = k)

print("Accuracies from %d individual folds:" % k)
print('scores', scores)

# kierrosten keskiarvo
print("Accuracy calculated using %d-fold cross validation = %.3f" % (k, scores.mean()))

Ristiinvalidointi

In [ ]:
from sklearn.model_selection import cross_val_score

k = 2
scores = cross_val_score(estimator = classifier,
                         X = X,
                         y = y,
                        scoring = 'accuracy',
                        cv = k)

print(k)
print(scores)
print(scores.mean())